In [ ]:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata
import os
from config2 import app_token

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
# client = Socrata("data.cityofchicago.org", none)

# Example authenticated client (needed for non-public datasets):
# client = Socrata("data.cityofchicago.org",
#                  "du5ugsgl0w1g4ql37x78pjbc5",
#                  username= "dr.harmony",
#                  password= "F3ar&Loathing")

# # Example authenticated client (needed for non-public datasets):
client = Socrata("data.cityofchicago.org",
                 app_token)
#                  username= username,
#                  password= password)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("85ca-t3if", limit = 500000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df

In [ ]:
results_df.columns

In [ ]:
crashes = results_df.drop(columns=['crash_record_id', 'location', 'injuries_unknown', 'injuries_incapacitating', 'injuries_fatal', 'injuries_no_indication', 'injuries_reported_not_evident', 'injuries_non_incapacitating', 'private_property_i', 'crash_date_est_i', 'statements_taken_i', 'photos_taken_i', 'work_zone_i', 'work_zone_type', 'workers_present_i', 'dooring_i'])
crashes

In [ ]:
crashes.rename(columns={"beat_of_occurrence": "Beat"}, inplace = True)

In [ ]:
crashes.dtypes

In [ ]:
crashes[["Beat"]] = crashes[["Beat"]].apply(pd.to_numeric)
crashes.dtypes

In [ ]:
# Reference the file where the CSV is located
neighborhood_path = "ChicagoAreasBeatDistricts.csv"

# Import the data into a Pandas DataFrame
neighborhoods_df = pd.read_csv(neighborhood_path)
neighborhoods_df.dtypes

In [ ]:
crash_neighborhood_merge = pd.merge(left=crashes, right=neighborhoods_df, how='right', on=['Beat'])
# crash_neighborhood_merge = pd.concat([crashes, neighborhoods_df], axis=1)
crash_neighborhood_merge_clean = crash_neighborhood_merge[['crash_date', 'posted_speed_limit', 'traffic_control_device',
       'device_condition', 'weather_condition', 'lighting_condition',
       'first_crash_type', 'trafficway_type', 'alignment',
       'roadway_surface_cond', 'road_defect', 'report_type', 'crash_type',
       'intersection_related_i', 'damage', 'date_police_notified',
       'prim_contributory_cause', 'sec_contributory_cause', 'street_no',
       'street_direction', 'street_name', 'District', 'Community Area', 'Beat', 'num_units',
       'most_severe_injury', 'injuries_total', 'crash_hour',
       'crash_day_of_week', 'crash_month', 'latitude', 'longitude',
       'hit_and_run_i', 'rd_no', 'lane_cnt']]
crash_neighborhood_merge_clean

In [ ]:
crash_neighborhood_merge_clean.columns

In [ ]:
# finalhtml = crash_neighborhood_merge_clean.to_html('crashes.html')